# Amazon Titan을 사용한 추상적 텍스트 요약

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Overview
대용량 문서로 작업할 때 입력 텍스트가 모델 컨텍스트 길이에 맞지 않거나, 모델이 대용량 문서를 인식하지 못하거나, 메모리 부족 오류 등으로 인해 몇 가지 문제에 직면할 수 있습니다.

이러한 문제를 해결하기 위해 청킹 및 연쇄 프롬프트 개념을 기반으로 하는 아키텍처를 보여드리겠습니다. 이 아키텍처는 언어 모델 기반 애플리케이션을 개발하는 데 널리 사용되는 프레임워크인 [LangChain](https://python.langchain.com/docs/get_started/introduction.html)을 활용합니다.

### Architecture

![](./images/42-text-summarization-2.png)

이 아키텍처에서는

- 대용량 문서(또는 작은 파일을 추가한 대용량 파일)가 로드됩니다.
- LangChain 유틸리티를 사용하여 여러 개의 작은 청크로 분할(청킹)합니다.
- 첫 번째 청크가 모델로 전송되고, 모델은 해당 요약을 반환합니다.
- 랭체인은 다음 청크를 가져와서 반환된 요약에 추가하고 결합된 텍스트를 모델에 새 요청으로 보냅니다. 이 과정은 모든 청크가 처리될 때까지 반복됩니다.
- 최종적으로 전체 콘텐츠를 기반으로 한 최종 요약이 생성됩니다.

### Use case
이 접근 방식은 통화 녹취록, 회의 녹취록, 책, 기사, 블로그 게시물 및 기타 관련 콘텐츠를 요약하는 데 사용할 수 있습니다.

## Setup

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 필요한 라이브러리가 설치되어 있는지 확인하고 베드락에 연결해야 합니다.

설정 방법과 ⚠️ **변경이 필요한지 여부**에 대한 자세한 내용은 [Bedrock 기본환경 설정 노트북](../00_Setup/setup.ipynb) 노트북을 참조하세요.

이 노트북에는 입력 프롬프트에서 토큰 수를 계산하는 데 사용할 [Hugging Face Transformers](https://huggingface.co/docs/transformers/index) 라이브러리도 설치합니다.

In [ ]:
import os
import sys
import json
import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

In [ ]:
is_internal_use = True # True: AWS, False: Client
#bedrock_region = "" ## <your region> 
bedrock_region = "us-west-2"

In [ ]:
if bedrock_region == "us-east-1":
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url": "https://bedrock.us-east-1.amazonaws.com" if is_internal_use else None
    }
elif bedrock_region == "us-west-2":
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url": "https://prod.us-west-2.frontend.bedrock.aws.dev" if is_internal_use else None
    }

In [ ]:
if is_internal_use:
    boto3_bedrock = boto3.client(
        service_name='bedrock',
        region_name=bedrock_config["region_name"],
        endpoint_url=bedrock_config["endpoint_url"]
    )
else:
    boto3_bedrock = boto3.client(
        service_name='bedrock',
        region_name=bedrock_config["region_name"]
    ) 

In [ ]:
%pip install --quiet langchain==0.0.249 "transformers>=4.24,<5"

## 긴 텍스트 요약 

### Boto3로 LangChain 구성하기

LangChain에 boto3 세션 정보를 전달하면 베드락에 액세스할 수 있습니다. LangChain에 boto3 세션 정보로 None을 전달하면 LangChain은 사용자 환경에서 세션 정보를 얻으려고 시도합니다.
올바른 클라이언트가 사용되도록 하기 위해 유틸리티 메서드를 사용하여 클라이언트를 인스턴스화할 것입니다.

LangChain Bedrock 클래스에 LLM을 지정해야 하며, 추론을 위한 인수를 전달할 수 있습니다. 여기서는 `model_id`에 Amazon Titan Text Large를 지정하고 `textGenerationConfig`에 Titan의 추론 파라미터를 전달합니다.

In [2]:
from langchain.llms.bedrock import Bedrock

llm = Bedrock(
    model_id="amazon.titan-tg1-large",
    model_kwargs={
        "maxTokenCount": 4096,
        "stopSequences": [],
        "temperature": 0,
        "topP": 1,
    },
    client=boto3_bedrock,
)

### 토큰이 많은 텍스트 파일 로드하기

`letters` 디렉토리에서 [2022년 주주들에게 보내는 아마존 CEO의 편지](https://www.aboutamazon.com/news/company-news/amazon-ceo-andy-jassy-2022-letter-to-shareholders) 텍스트 파일을 찾을 수 있습니다. 다음 셀은 텍스트 파일을 로드하고 파일에 있는 토큰 수를 계산합니다. 

텍스트 파일의 토큰 수가 해당 모델의 최대 토큰 수를 초과한다는 경고가 표시됩니다.

In [3]:
shareholder_letter = "./letters/2022-letter.txt"

with open(shareholder_letter, "r") as file:
    letter = file.read()
    
llm.get_num_tokens(letter)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Token indices sequence length is longer than the specified maximum sequence length for this model (6526 > 1024). Running this sequence through the model will result in indexing errors


6526

### 긴 텍스트를 청크단위로 나누기

텍스트가 너무 길어서 프롬프트에 맞지 않으므로 작은 청크로 분할합니다.
LangChain의 `RecursiveCharacterTextSplitter`는 각 청크의 크기가 `chunk_size`보다 작아질 때까지 긴 텍스트를 재귀적으로 청크로 분할하는 것을 지원합니다. 텍스트는 `separators=["\n\n", "\n"]`를 사용하여 청크로 분리되므로 각 단락이 여러 청크로 분할되는 것을 방지할 수 있습니다.

청크당 6,000자를 사용하면 각 부분에 대한 요약을 개별적으로 얻을 수 있습니다. 청크에 포함된 토큰 또는 단어 조각의 수는 텍스트에 따라 다릅니다.

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], chunk_size=4000, chunk_overlap=100
)

docs = text_splitter.create_documents([letter])

In [5]:
num_docs = len(docs)

num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

print(
    f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens"
)

Now we have 10 documents and the first one has 439 tokens


### 청크 요약 및 결합하기

다른 문서에서 토큰 수가 일정하다고 가정하면 문제가 없을 것입니다. LangChain의 [load_summarize_chain](https://python.langchain.com/en/latest/use_cases/summarization.html)을 사용하여 텍스트를 요약해 보겠습니다. `load_summarize_chain`은 `stuff`, `map_reduce`, `refine`의 세 가지 요약 방법을 제공합니다. 
- `stuff`는 모든 청크를 하나의 프롬프트에 넣습니다. 따라서 토큰의 최대 한도에 도달하게 됩니다.
- `map_reduce`는 각 청크를 요약하고 요약을 결합한 후 결합된 요약을 요약합니다. 결합된 요약이 너무 크면 오류가 발생합니다.
- `refine`은 첫 번째 청크를 요약한 다음 첫 번째 요약과 함께 두 번째 청크를 요약합니다. 모든 청크가 요약될 때까지 동일한 프로세스가 반복됩니다.

`map_reduce`와 `refine`은 LLM을 여러 번 호출하기 때문에 최종 요약을 얻는 데 시간이 걸립니다. 
여기서는 `map_reduce`를 사용해 보겠습니다.

In [6]:
# Set verbose=True if you want to see the prompts being used
from langchain.chains.summarize import load_summarize_chain
summary_chain = load_summarize_chain(llm=llm, chain_type="map_reduce", verbose=False)

> ⏰ **노트:** 문서 수, 베드락 요청 속도 할당량 및 구성된 재시도 설정에 따라 이 체인을 실행하는 데 다소 시간이 걸릴 수 있습니다:

In [ ]:
output = summary_chain.run(docs)

In [ ]:
print_ww(output.strip())